In [1]:
###########import packages##########
import tensorflow as tf
import keras
from keras import optimizers
from keras import regularizers
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras.constraints import max_norm
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import Dropout 
from keras.models import Model
from keras.layers import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.wrappers.scikit_learn import KerasRegressor
from keras.constraints import maxnorm 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb
import lightgbm
import catboost
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from xgboost import plot_importance
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import ensemble
from sklearn.tree import ExtraTreeRegressor
from sklearn import svm
from sklearn import neighbors
from sklearn import tree
from sklearn.impute import SimpleImputer
from fancyimpute import KNN
import keras_metrics as km
from keras.callbacks import EarlyStopping 

early_stopping=keras.callbacks.EarlyStopping(
 monitor="val_loss", 
 patience=20, 
 verbose=0, 
 mode="auto"
)
%matplotlib
###########loading data##########
fdata=pd.read_csv('database_filled.csv',encoding="gbk")
raw_data=fdata.loc[:,[                      
                      'Fe Cotent (wt. %)',#2
                      'O Content (at. %)',#4
                      'Content of Pyridinic Species (at. %)',#5
                      'Raman ID/IG Ratio',#8
                      'BET Surface Area (m2 g-1)',#9
                      'Micropore Ratio',#10
                      'Mesopore Ratio',#11
                      'Catalyst Loading on RDE/RRDE(mg cm-2)',
                      'HW_0.78'#the classification target#14
                        ]]

###########data standardization##########
standardized_data = (raw_data-np.mean(raw_data,axis=0))/np.std(raw_data,axis=0)

###########defining a wrapper function for later call from each machine learning algorithms##########
raw_input=standardized_data.iloc[:,0:8]
raw_output=raw_data.iloc[:,8]
###########fix random seed for reproducability##########
seed=117
###########train test splitting##########
X_train, X_test, y_train, y_test = train_test_split(raw_input, raw_output, test_size=.15,random_state=seed)
raw_input_global=raw_data.iloc[:,0:8]
raw_output_global=raw_data.iloc[:,8]
###########wrap up fuction for later call for OPTIMIZATION##########
def evaluate(pre_2,real_2):
    pre_2=np.array(pre_2)
    real_2=np.array(real_2)
    pre_2_series=pd.Series(pre_2)
    real_2_series=pd.Series(real_2)
    return rmse(pre_2,real_2), round(pre_2_series.corr(real_2_series), 3)
def compare(list_name,limit):
    judge=1
    for a in list_name:
        if a < limit:
            judge=judge*1
        else:
            judge=judge*0
    return judge
def generate_arrays_from_file(path):
    while True:
        with open(path) as f:
            for line in f:
                # create numpy arrays of input data
                # and labels, from each line in the file
                x1, x2, y = process_line(line)
                yield ({'input_1': x1, 'input_2': x2}, {'output': y})
def intergate(y_pred):
    length=y_pred.shape[0]
    print(length)
    for i in range (0,length):
        if y_pred[i][0]>=0.5:
            y_pred[i][0]=1
        else:
            y_pred[i][0]=0
    return y_pred
accuracy={}
for neurons1 in [50,100,200]:
    for dropout_rate in [0,0.25,0.5]:
        for batch_size_number in [16,32]:
            for reg in [0,0.0001,0.001]:
                for act in ['sigmoid','tanh','relu','softsign']:                        
                    for epochs_number in [200,400,800]:
                        regularizer=keras.regularizers.l2(reg)
                        ###########keras ANN model construction########## 
                        model = Sequential() 
                        model.add(Dense(neurons1, input_dim=8, kernel_initializer='random_normal',
                                        bias_initializer='random_normal',activation='tanh',kernel_regularizer=regularizer)) 
                        model.add(Dropout(dropout_rate))                        
                        model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                                        bias_initializer='random_normal',activation=act,kernel_regularizer=regularizer)) 
                        model.add(Dropout(dropout_rate))
                        model.add(Dense(1, input_dim=neurons1, activation='sigmoid'))
                        model.compile(loss='binary_crossentropy', optimizer='rmsprop',metrics=[km.f1_score(), km.binary_precision(), km.binary_recall()])
                        print('training...')
                        model.fit(X_train, y_train,verbose=0, batch_size=batch_size_number,epochs=epochs_number,validation_split=0.2,callbacks=[early_stopping])
                        print(neurons1,epochs_number,dropout_rate,batch_size_number,reg,act)
                        y_pred_test=model.predict(X_test)
                        y_pred_train=model.predict(X_train)
                        y_pred_test=intergate(y_pred_test)
                        y_pred_train=intergate(y_pred_train)
                        print(classification_report(y_train,y_pred_train))
                        print(classification_report(y_test,y_pred_test))
                        final_result=classification_report(y_test,y_pred_test,output_dict=True)
                        ac=final_result['accuracy']
                        accuracy[ac]=[neurons1,epochs_number,dropout_rate,batch_size_number,reg,act]
                        K.clear_session()    

Using matplotlib backend: Qt5Agg
training...


C:\Users\Administratro\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\base_layer.py:1402: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '


C:\Users\Administratro\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\base_layer.py:1402: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '


50 200 0 16 0 sigmoid
34
191
              precision    recall  f1-score   support

           0       0.65      0.78      0.71       104
           1       0.66      0.51      0.57        87

    accuracy                           0.65       191
   macro avg       0.65      0.64      0.64       191
weighted avg       0.65      0.65      0.65       191

              precision    recall  f1-score   support

           0       0.80      0.73      0.76        22
           1       0.57      0.67      0.62        12

    accuracy                           0.71        34
   macro avg       0.69      0.70      0.69        34
weighted avg       0.72      0.71      0.71        34

training...


C:\Users\Administratro\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\base_layer.py:1402: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '


C:\Users\Administratro\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\base_layer.py:1402: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '


KeyboardInterrupt: 

In [ ]:
print(accuracy)

In [ ]:
0.8235294117647058: [200, 400, 0, 32, 0, 'softsign']

In [ ]:
import eli5
def base_model():
    tmodel = Sequential() 
    tmodel.add(Dense(200, input_dim=8, kernel_initializer='random_normal',
                    bias_initializer='random_normal',activation='tanh',kernel_regularizer=keras.regularizers.l2(0))) 
    tmodel.add(Dropout(0))
    tmodel.add(Dense(200, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='softsign',kernel_regularizer=keras.regularizers.l2(0))) 
    tmodel.add(Dropout(0))
    tmodel.add(Dense(1, input_dim=200, activation='sigmoid'))
    tmodel.compile(loss='binary_crossentropy', optimizer='rmsprop',metrics=['accuracy'])
    return tmodel
from eli5.sklearn import PermutationImportance
my_model = KerasClassifier(build_fn=base_model,nb_epoch=400, batch_size=32, verbose= False)    
my_model.fit(X_train, y_train)
perm = PermutationImportance(my_model, random_state=1,n_iter=10).fit(X_train,y_train)
eli5.show_weights(perm)

In [2]:
###########keras ANN model construction##########
smodel = Sequential() 
smodel.add(Dense(200, input_dim=8, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='tanh',kernel_regularizer=keras.regularizers.l2(0))) 
smodel.add(Dropout(0))
smodel.add(Dense(200, input_dim=neurons1, kernel_initializer='random_normal',
            bias_initializer='random_normal',activation='softsign',kernel_regularizer=keras.regularizers.l2(0))) 
smodel.add(Dropout(0))
smodel.add(Dense(1, input_dim=200, activation='sigmoid'))
smodel.compile(loss='binary_crossentropy', optimizer='rmsprop',metrics=['accuracy'])
print('training...')
smodel.fit(X_train, y_train,verbose=0, batch_size=32,epochs=400)

training...


In [3]:
import shap
# %matplotlib
SHAP_INPUT=standardized_data.iloc[:,0:8]
SHAP_OUTPUT=standardized_data.iloc[:,8]
X_SHAP=SHAP_INPUT.values.astype(np.float32)
y_SHAP=SHAP_OUTPUT.values.astype(np.float32)
explainer = shap.GradientExplainer(smodel,X_SHAP)
cols=[               'Fe Cotent (wt. %)',#2
                      'O Content (at. %)',#4
                      'Content of Pyridinic Species (at. %)',#5
                      'Raman ID/IG Ratio',#8
                      'BET Surface Area (m2 g-1)',#9
                      'Micropore Ratio',#10
                      'Mesopore Ratio',#11
                      'Catalyst Loading on RDE/RRDE(mg cm-2)'
                        ]
shap_values = explainer.shap_values(X_SHAP)
print(type(shap_values))

`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


<class 'list'>


In [4]:
shap.summary_plot(shap_values[0], SHAP_INPUT)

In [7]:
shap.dependence_plot('Fe Cotent (wt. %)', shap_values[0], SHAP_INPUT, interaction_index=None, show=False)